### CARREGAR DADOS

In [14]:
import pandas as pd
import numpy as np

In [15]:
# load reddit data

comments = pd.read_csv('raw/comments_all.csv')

submissions = pd.read_csv('raw/submissions_all.csv')

print('Comments shape:', comments.shape)
print('Submissions shape:', submissions.shape)

print ('Comments columns:', comments.columns.tolist())
print ('Submissions columns:', submissions.columns.tolist())

/tmp/ipykernel_6971/2811161009.py:3: DtypeWarning: Columns (11,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  comments = pd.read_csv('raw/comments_all.csv')
/tmp/ipykernel_6971/2811161009.py:5: DtypeWarning: Columns (10,19,28) have mixed types. Specify dtype option on import or set low_memory=False.
  submissions = pd.read_csv('raw/submissions_all.csv')


Comments shape: (7348257, 18)
Submissions shape: (390924, 32)
Comments columns: ['id', 'author', 'subreddit', 'subreddit_id', 'created_utc', 'is_submitter', 'body', 'controversiality', 'link_id', 'parent_id', 'score', 'send_replies', 'total_awards_received', 'stickied', 'archived', 'collapsed_reason', 'utc_datetime_str', 'created_timestamp_utc']
Submissions columns: ['id', 'author', 'author_fullname', 'title', 'url', 'selftext', 'link_flair_text', 'over_18', 'permalink', 'num_crossposts', 'crosspost_parent', 'name', 'quarantine', 'archived', 'removed_by_category', 'score', 'stickied', 'author_cakeday', 'banned_by', 'media', 'is_video', 'view_count', 'media_only', 'subreddit', 'subreddit_id', 'num_comments', 'upvote_ratio', 'total_awards_received', 'suggested_sort', 'created_utc', 'retrieved_utc', 'created_timestamp_utc']


In [16]:
comments[['body','id']]


,body,id
0,clássico. \n\nfeliz ano novo,hqqv2hg
1,Onde vejo a tabela de classificação de classe?,hqqv55q
2,[deleted],hqqv5gu
3,Ha tem muita gente falando exatamente isso campeão,hqqv5sn
4,"Eu nem consegui apostar. Todas as lotéricas que eu fui essa semana estavam ""sem sistema"" e o app... bons todo mundo sabe",hqqv5y1
...,...,...
7348252,Só nos primeiros dias. Não se animem,j2fxd4p
7348253,[Nos cinco primeiros minutos ele explica tudo](https://youtu.be/ksMaA-GcsL0),j2fxg2i
7348254,"Realmente, Roberta também tem culpa",j2fxk5x
7348255,"Sim op, sim! Eu também amo ver musculosos fardados com armas de grande porte.",j2fxl62


In [17]:
submissions[['selftext', 'id']]

,selftext,id
0,NaN,rt9yre
1,"Esse post é para quem trocou a noite pelo dia, quer jogar conversa fora, mas não tem uma postagem legal para fazer no momento. Vamos bater um papo.",rt9yyx
2,[removed],rt9zb5
3,Só tem os bots postando aqui esse ano...,rt9zoz
4,"Parabéns galera, feliz ano novo!",rta22e
...,...,...
390919,[deleted],1006gqb
390920,NaN,1006ikp
390921,"A falha foi minha. Quero mandar um abraço a todos vocês e desejar boa sorte. Eu estou fora, tá bom? Vida que segue",1006mpo
390922,[deleted],1006ob0


### FILTRAGEM INICIAL


In [18]:
import re
import unicodedata
from unidecode import unidecode
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords', quiet=True)
stopwords_pt = set(stopwords.words('portuguese'))

In [19]:
def tokenize(text):


    text = text.lower() # lowercase

    text = unidecode(text) # remove accents

    text = re.sub(r'[^0-9A-Za-z\s]', '', text) # remove special characters

    text = re.sub(r'\s+', ' ', text).strip()  # remove extra spaces

    tokenized_text = text.split() # split into words

    tokenized_text = [word for word in tokenized_text if word not in stopwords_pt]   # remove stopwords
    
    tokenized_text = [word for word in tokenized_text if len(word) > 2] # remove short words

    return tokenized_text


print(tokenize("OLÀ, tudo bem? \n\nEu sou um texto com caracteres unicode: ñ, é, ç, ü, jão, etc."))

# convert 'body' and 'selftext' columns to string type to avoid errors during tokenization
comments['body'] = comments['body'].astype(str)
submissions['selftext'] = submissions['selftext'].astype(str)

# tokenize the 'body' column in comments and 'selftext' column in submissions
comments['tokenized_body'] = comments['body'].apply(tokenize)
submissions['tokenized_selftext'] = submissions['selftext'].apply(tokenize)
  

['ola', 'tudo', 'bem', 'texto', 'caracteres', 'unicode', 'jao', 'etc']


In [20]:
def contains_seed_words(tokenized_text, seed_words, min_count=2):
    necessary_words = {
        'ucrania', 'ucraniano', 'ucraniana', 'ucranianos', 'ucranianas',
        'russia', 'russa', 'russo', 'russos', 'russas',
        'putin', 'zelensky', 'zelenskyy', 'zelenskiy', 'zelenskyi',
        'zelenski', 'zelenskyj','crimeia', 'crimea','kiev', 'kyiv', 'kievs', 'kyivs',
    }
    
    necessary_count = sum(1 for w in tokenized_text if w in necessary_words)

    if necessary_count >= min_count:
        return True
    
    if necessary_count == 1:

        if any(w in seed_words for w in tokenized_text):
            return True
        else:
            return False
    

    return False

In [21]:

seed_words = [
    'guerra', 'guerras','conflito', 'conflitos','invasao', 'invasoes','tropa', 'tropas','nato', 'otan','sancao',
    'sancoes', 'embargo', 'embargos','bombardeio', 'bombardeios','ocupacao', 'ocupacoes','resistencia','ofensiva', 'ofensivas','defesa', 'defesas',
    'tanque', 'tanques', 'blindado', 'blindados','cessar-fogo', 'cessar_fogo','diplomacia','negociacao', 'negociacoes',
    'dialogo','paz','refugiado', 'refugiada', 'refugiados', 'refugiadas','donetsk','war'
    'luhansk','mariupol','ue', 'uniao europeia','europeu', 'europeia', 'europeus', 'europeias'
]

# seed words to filter comments
comments['contains_seed_words'] = comments['tokenized_body'].apply(lambda x: contains_seed_words(x, seed_words))
submissions['contains_seed_words'] = submissions['tokenized_selftext'].apply(lambda x: contains_seed_words(x, seed_words))

# filter submissions based on the presence of seed words
filtered_submissions = submissions[submissions['contains_seed_words']]
# filter comments based on the presence of seed words
filtered_comments = comments[comments['contains_seed_words']]

print('Filtered Submissions shape:', filtered_submissions.shape)
print('Filterad Comments shape:', filtered_comments.shape)


Filtered Submissions shape: (551, 34)
Filterad Comments shape: (28434, 20)


In [22]:
pd.set_option('display.max_colwidth', None)
filtered_submissions[['selftext']].iloc[0:5]

,selftext
3953,\- Apoio a Russia. Lutaria se pedissem. \n\- Apoio a Otan. Lutaria se pedissem. \n\- Apoio a Russia. Fugiria se houver convocação. \n\- Apoio a Otan. Fugiria se houver convocação. \n\- Movimento Brasil Neutro e mais Independente do Globalismo. \n\nSe você está por fora:\n\n[https://www.youtube.com/watch?v=7\_kbLfp8LVQ&amp;ab\_channel=HojenoMundoMilitar](https://www.youtube.com/watch?v=7_kbLfp8LVQ&amp;ab_channel=HojenoMundoMilitar)\n\n[https://www.youtube.com/watch?v=De86Q\_mpyDI&amp;ab\_channel=GlobalNews](https://www.youtube.com/watch?v=De86Q_mpyDI&amp;ab_channel=GlobalNews)\n\n[https://www.youtube.com/watch?v=x4ZYgugPOVY&amp;ab\_channel=CBCNews](https://www.youtube.com/watch?v=x4ZYgugPOVY&amp;ab_channel=CBCNews)\n\n[https://www.youtube.com/watch?v=7XZc0qdlVhY&amp;ab\_channel=UATVEnglish](https://www.youtube.com/watch?v=7XZc0qdlVhY&amp;ab_channel=UATVEnglish)\n\n[https://www1.folha.uol.com.br/mundo/2022/01/russia-envia-tropas-a-belarus-para-exercicios-proximos-a-fronteira-com-a-ucrania.shtml](https://www1.folha.uol.com.br/mundo/2022/01/russia-envia-tropas-a-belarus-para-exercicios-proximos-a-fronteira-com-a-ucrania.shtml)\n\n[https://www.cnnbrasil.com.br/internacional/alemanha-pode-interromper-oleoduto-se-russia-atacar-ucrania-sinaliza-scholz/](https://www.cnnbrasil.com.br/internacional/alemanha-pode-interromper-oleoduto-se-russia-atacar-ucrania-sinaliza-scholz/)\n\n[https://g1.globo.com/mundo/noticia/2022/01/18/reino-unido-e-canada-enviam-ajuda-militar-a-ucrania-em-meio-a-aumento-de-tensoes-com-a-russia.ghtml](https://g1.globo.com/mundo/noticia/2022/01/18/reino-unido-e-canada-enviam-ajuda-militar-a-ucrania-em-meio-a-aumento-de-tensoes-com-a-russia.ghtml)
4088,"Olhem o vídeo, por exemplo, de uma matéria falando da Suécia. ([https://www.youtube.com/watch?v=sxnxwvDvdCA](https://www.youtube.com/watch?v=sxnxwvDvdCA))\n\nUm bando de gente que fica falando palavras de ordem típica de propaganda, destilando preconceitos e esteriótipos (como se a Suécia fosse só a Greta Thumberg, e acolher refugiados fosse um demérito pra quem não teve a cabeça frita pela armadilha dos memes do Pepe o sapo e outras coisas da extrema direita). \n\nE isso acontece não só em canais pró Rússia ou pró Putin (saibam separar o que é ódio a Putin como uma ferramenta de oligopólios e pessoas economicamente poderosas de seu país, do ódio a Rússia como país, seu povo, sua língua, seus costumes e tradições), mas por toda a internet, forçando nostalgia soviética, vaporwave em cima de música soviética, glorificando uma nação que já não existe mais... Me irrita e muito essa idolatria cega, como se qualquer russo padrão desse a mínima para o que o Brasil está fazendo, se a gente está passando fome ou prosperando.\n\nE isso acontece porque hoje em dia, não há identidade nacional. Aqui no Brasil não. Nem em países onde ainda há isso, como os EUA e países europeus, se vê uma renúncia disso a um padrão soviético, com lindos uniformes militares impecáveis, armas limpas e lustradas, marchas precisas e músicas inspiradoras, coisa que essa turma nunca nem viu de perto. Idolatram algo que nunca viram e nem poderão ver. É a premissa do vaporwave mesmo - a saudade de algo que nunca se viveu.\n\nNão vou aqui falar que isso é obra do governo russo, mas, é certamente negativo e contraprodutivo esse tanto de gente tendo uma ideia de estimação que nem existe mais, ou, que é completamente falsa."
4277,"Só pra saber q eu n sou o único vendo isso!? mas vcs repararam que ta praticamente certo uma invasão russa na Ucrânia? E tipo parece um jogo de pôquer, e ta todo mundo jogando alto, EUA mandou "" CÊ N TEM CORAGEM!"" pra Rússia, a Alemanha que costuma ser ""pau mandado"" da Russia falou ""METE O LOCO NÃO!"" e a Rússia continua colocando tropa na fronteira e criando um ambiente tenso entre os Brothers, honestamente pelo que vi, só existem duas possibilidades, guerra ou essa tensão vai diminuir mas continuar existindo, pois o motivo inicial pra ela é razoável pra ambos os 

In [23]:
filtered_comments[['body']].iloc[0:5]  

,body
3190,"Eu não coloco a tua experiência em causa, mas é só isso, uma experiência que vale muito pouco e que é facilmente contradita com factos. E o que escreveste demonstra uma enorme ignorância e falta de sensatez. \n\nDeixa-me começar por aqui: \n\n&gt; Sobre a Suécia não tenho nada a comentar, aquele lugar é quase um paraíso na terra.\n\nTens noção do quão patético isto é? A Suécia foi dos primeiros países a assustar a UE com o rápido crescimento da extrema direita e movimentos anti-imigração. \n\nTiveste uma boa experiência na Suécia. Perfeito. Eu também e já lá fui várias vezes. Também conheço portugueses (e os seus amigos sul europeus) que lá fizeram fizeram estudos pós-graduados e chegaram a ser diretamente mandados para a terra deles de forma agressiva.\n\nMas não interessa nada a experiência de cada um. O que se sabe é que de facto existe fortes movimentos de extrema direita, anti-imigração na Suécia. Não é em Portugal, é na Suécia. \n\n&gt; De início eu imaginava que em Portugal a população seria mais 'receptiva' do que nos EUA, isso porque o Brasil tem algumas similaridades com Portugal e pelo fato de os americano\n\nO que significa zero. Não tens uma enorme vaga migratória dos Brasil para os EUA e um brasileiro em cada canto neste país (nem na Suécia) como tens em Portugal, e só assim a comparação faria sentido. \n\nPara não dizer que os EUA é um país gigante, muito pouco uniforme, a tua experiência vai variar imenso dependendo de para onde vais, e ninguém sabe ou ouve quase nada sobre o Brasil. Rigorosamente nada disso significa que os EUA tenham sejam menos xenófobos que Portugueses. E tu ao falares no México mostras que sabes muito bem que não são. \n\nAgora facto é que eles elegeram e quase re-elegeram Trump (Portugal estava no topo dos países europeus com maior rejeição de Trump). \n\nFacto é que a população do Brasil elegeu um Bolsonaro, abertamente racista, que fala de uma forma mais explícita e direta do que André Ventura, o líder do pequeno partido de extrema direita português. \n\nFacto é que Portugal não tem nenhum movimento anti-imigração. Não houve no ano 2000 quando num ano passamos do 0 para 60K ucranianos. Não houve com as vagas de migrações brasileiras anteriores nem com esta atual. Encontras brasileiros em todo o lado e é um não assunto em Portugal. Nem o líder da extrema direita alguma vez abriu a boca contra migração brasileira. \n\nSabes que já ratificámos o tratado para facilitar imigração de outros países lusófonos (ao contrário de vocês). Viste a contestação (não viste, não existiu).\n\nFacto é que Portugal, com os preconceitos que possa ter, elegeu um líder bem escurinho, de descendência Goesa. \n\nFacto é que Portugal consome sem qualquer preconceito tudo o que é cultura brasileira (ao contrário de vocês com a nossa), que tem presença regular de brasileiros na tv portuguesa ou de brasileiros a escrever em publicações portuguesas (em português do brasil). \n\nNão duvido nada que tenhas ouvido coisas desagradáveis de alguns idiotas em Portugal. Mas isso não muda que Portugal não é como o Reino Unido onde as notícias sobre a sua relação com as maiores comunidades de imigrantes, como os Polacos, volta e meia lá envolvem mais um assassinato ou suicídio por causa de bullying. (Mas puta que me pariu se não vou continuar a ouvir como o UK é que é cosmopolita e tolerante ao contrário de Portugal... completa falta de noção)."
3191,"&gt;Farmaajo tem sido apoiado por muito tempo por Washington por lutar ao lado do Comando Africano dos EUA (USAFRICOM, na sigla em inglês) contra o Al-Shabaab, um grupo ligado à Al-Qaeda (organização terrorista proibida na Rússia e em vários outros países), e por avançar os interesses de petróleo dos EUA e do Reino Unido na região, interrompidos por anos pela guerra civil no país africano."
3575,"O que é dar certo? Porque pensa bem, o país era basicamente um feudo até 1916, ai rolou a revolução e em 1930 eles já eram uma potencia industrial, em 1945 derrotaram a maior

### TOP PALAVRAS MAIS FREQUENTES NA FILTRAGEM INICIAL

In [24]:
token_tf = {}

# calculate term frequency for each token in the filtered comments and submissions
for index, row in filtered_comments.iterrows():
    for token in row['tokenized_body']:
        if token not in token_tf:
            token_tf[token] = 0
        token_tf[token] += 1

for index, row in filtered_submissions.iterrows():
    for token in row['tokenized_selftext']:
        if token not in token_tf:
            token_tf[token] = 0
        token_tf[token] += 1    

# sort tokens by their term frequency
sorted_tokens = sorted(token_tf.items(), key=lambda x: x[1], reverse=True)

# top K tokens
K = 200
top__tokens = sorted_tokens[:K]

top__tokens = [(token, freq) for token, freq in top__tokens if token not in seed_words]

print(top__tokens)

[('nao', 46564), ('russia', 26320), ('ucrania', 20337), ('pra', 14823), ('putin', 9883), ('pais', 8181), ('vai', 8180), ('voce', 7785), ('sao', 7616), ('eua', 7551), ('paises', 5795), ('ate', 5509), ('mundo', 5390), ('brasil', 5348), ('tambem', 5228), ('ter', 5176), ('contra', 4890), ('porque', 4758), ('russo', 4743), ('russos', 4593), ('sobre', 4455), ('bem', 4381), ('estao', 4281), ('governo', 4229), ('ainda', 4120), ('fazer', 3999), ('agora', 3977), ('pode', 3852), ('russa', 3625), ('tudo', 3532), ('sim', 3509), ('nada', 3494), ('todo', 3369), ('anos', 3278), ('parte', 3278), ('entao', 3219), ('gente', 3218), ('assim', 3162), ('qualquer', 3129), ('china', 3124), ('coisa', 3123), ('acho', 3062), ('pessoas', 3038), ('aqui', 3023), ('cara', 2881), ('ucraniano', 2826), ('sendo', 2803), ('ucranianos', 2771), ('todos', 2748), ('faz', 2706), ('outros', 2673), ('ver', 2632), ('europa', 2614), ('povo', 2566), ('quer', 2562), ('outro', 2559), ('the', 2532), ('tao', 2509), ('lado', 2411), ('po

In [25]:
new_words = [
    'pais','eua', 'mundo', 'paises', 'governo', 'area','poder', 'estado', 'europa', 'povo', 'historia', 'historica', 'historico', 'populacao', 'presidente', 'ocidente', 'militar', 'exercito', 'militares', 'invadir', 'conquista',
    'conquistar', 'territorio', 'armas', 'arma', 'propaganda', 'nuclear', 'nucleares', 'economia', 'mundial', 'estados'
]

# merge seed words and new words
final_words = set(seed_words + new_words)

print(len(final_words))

# filter comments and submissions again based on the updated seed words

comments['contains_seed_words'] = comments['tokenized_body'].apply(lambda x: contains_seed_words(x, final_words))
submissions['contains_seed_words'] = submissions['tokenized_selftext'].apply(lambda x: contains_seed_words(x, final_words))

final_filtered_comments = comments[comments['contains_seed_words']]
final_filtered_submissions = submissions[submissions['contains_seed_words']]

print('Filtered Comments shape after merging new words:', final_filtered_comments.shape)
print('Filtered Submissions shape after merging new words:', final_filtered_submissions.shape)

78
Filtered Comments shape after merging new words: (36854, 20)
Filtered Submissions shape after merging new words: (729, 34)


### VALIDACAO MANUAL

In [26]:
filtered_submissions[['selftext']].iloc[0:20]

,selftext
3953,\- Apoio a Russia. Lutaria se pedissem. \n\- Apoio a Otan. Lutaria se pedissem. \n\- Apoio a Russia. Fugiria se houver convocação. \n\- Apoio a Otan. Fugiria se houver convocação. \n\- Movimento Brasil Neutro e mais Independente do Globalismo. \n\nSe você está por fora:\n\n[https://www.youtube.com/watch?v=7\_kbLfp8LVQ&amp;ab\_channel=HojenoMundoMilitar](https://www.youtube.com/watch?v=7_kbLfp8LVQ&amp;ab_channel=HojenoMundoMilitar)\n\n[https://www.youtube.com/watch?v=De86Q\_mpyDI&amp;ab\_channel=GlobalNews](https://www.youtube.com/watch?v=De86Q_mpyDI&amp;ab_channel=GlobalNews)\n\n[https://www.youtube.com/watch?v=x4ZYgugPOVY&amp;ab\_channel=CBCNews](https://www.youtube.com/watch?v=x4ZYgugPOVY&amp;ab_channel=CBCNews)\n\n[https://www.youtube.com/watch?v=7XZc0qdlVhY&amp;ab\_channel=UATVEnglish](https://www.youtube.com/watch?v=7XZc0qdlVhY&amp;ab_channel=UATVEnglish)\n\n[https://www1.folha.uol.com.br/mundo/2022/01/russia-envia-tropas-a-belarus-para-exercicios-proximos-a-fronteira-com-a-ucrania.shtml](https://www1.folha.uol.com.br/mundo/2022/01/russia-envia-tropas-a-belarus-para-exercicios-proximos-a-fronteira-com-a-ucrania.shtml)\n\n[https://www.cnnbrasil.com.br/internacional/alemanha-pode-interromper-oleoduto-se-russia-atacar-ucrania-sinaliza-scholz/](https://www.cnnbrasil.com.br/internacional/alemanha-pode-interromper-oleoduto-se-russia-atacar-ucrania-sinaliza-scholz/)\n\n[https://g1.globo.com/mundo/noticia/2022/01/18/reino-unido-e-canada-enviam-ajuda-militar-a-ucrania-em-meio-a-aumento-de-tensoes-com-a-russia.ghtml](https://g1.globo.com/mundo/noticia/2022/01/18/reino-unido-e-canada-enviam-ajuda-militar-a-ucrania-em-meio-a-aumento-de-tensoes-com-a-russia.ghtml)
4088,"Olhem o vídeo, por exemplo, de uma matéria falando da Suécia. ([https://www.youtube.com/watch?v=sxnxwvDvdCA](https://www.youtube.com/watch?v=sxnxwvDvdCA))\n\nUm bando de gente que fica falando palavras de ordem típica de propaganda, destilando preconceitos e esteriótipos (como se a Suécia fosse só a Greta Thumberg, e acolher refugiados fosse um demérito pra quem não teve a cabeça frita pela armadilha dos memes do Pepe o sapo e outras coisas da extrema direita). \n\nE isso acontece não só em canais pró Rússia ou pró Putin (saibam separar o que é ódio a Putin como uma ferramenta de oligopólios e pessoas economicamente poderosas de seu país, do ódio a Rússia como país, seu povo, sua língua, seus costumes e tradições), mas por toda a internet, forçando nostalgia soviética, vaporwave em cima de música soviética, glorificando uma nação que já não existe mais... Me irrita e muito essa idolatria cega, como se qualquer russo padrão desse a mínima para o que o Brasil está fazendo, se a gente está passando fome ou prosperando.\n\nE isso acontece porque hoje em dia, não há identidade nacional. Aqui no Brasil não. Nem em países onde ainda há isso, como os EUA e países europeus, se vê uma renúncia disso a um padrão soviético, com lindos uniformes militares impecáveis, armas limpas e lustradas, marchas precisas e músicas inspiradoras, coisa que essa turma nunca nem viu de perto. Idolatram algo que nunca viram e nem poderão ver. É a premissa do vaporwave mesmo - a saudade de algo que nunca se viveu.\n\nNão vou aqui falar que isso é obra do governo russo, mas, é certamente negativo e contraprodutivo esse tanto de gente tendo uma ideia de estimação que nem existe mais, ou, que é completamente falsa."
4277,"Só pra saber q eu n sou o único vendo isso!? mas vcs repararam que ta praticamente certo uma invasão russa na Ucrânia? E tipo parece um jogo de pôquer, e ta todo mundo jogando alto, EUA mandou "" CÊ N TEM CORAGEM!"" pra Rússia, a Alemanha que costuma ser ""pau mandado"" da Russia falou ""METE O LOCO NÃO!"" e a Rússia continua colocando tropa na fronteira e criando um ambiente tenso entre os Brothers, honestamente pelo que vi, só existem duas possibilidades, guerra ou essa tensão vai diminuir mas continuar existindo, pois o motivo inicial pra ela é razoável pra ambos os 

### SAVE A CSV

In [29]:
# save the final filtered data to CSV files
final_filtered_comments.drop(columns=['contains_seed_words']).to_csv('comments_filtered.csv', index=False)
final_filtered_submissions.drop(columns=['contains_seed_words']).to_csv('submissions_filtered.csv', index=False)